# Conception d'un agent conversationnel spécialisé

## Scraper le site Ecofin

In [1]:
# url à scraper 
url="https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"
root="https://www.agenceecofin.com"

### Consultation des fichier roboot txt des sites 

In [3]:
import requests
from urllib.parse import urljoin, urlparse

def get_robots_txt(url):
        """Récupère le contenu du fichier robots.txt"""
        try:
            parsed_url = urlparse(url)
            name = parsed_url.netloc
            # Construire l'URL du robots.txt
            if not url.startswith(('http://', 'https://')):
                url = 'https://' + url
            robots_url = urljoin(url, '/robots.txt')
            
            # Faire la requête
            response = requests.get(robots_url)
            
            if response.status_code == 200:
                with open(f"robot_{name}.txt", "w") as file:
                    file.write(response.text)
                print("fichier crée")
            else:
                print(f"Erreur {response.status_code} pour {robots_url}")
               
        except Exception as e:
            print(f"Erreur pour {url}: {str(e)}")

In [4]:
get_robots_txt(root)

fichier crée


### Extraction des articles

In [ ]:
# scraper le site 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import json
import time

# Configuration du navigateur Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Exécuter sans interface graphique
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 🔹 URL de départ
start_url = "https://www.agenceecofin.com/a-la-une/recherche-article?filterTitle=&submit.x=0&submit.y=0&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=&filterDateTo=&option=com_dmk2articlesfilter&view=articles&filterFrench=French&Itemid=269&userSearch=1&layout=#dmk2articlesfilter_results"  # Remplace avec l'URL du site
driver.get(start_url)
time.sleep(8)  # Attendre le chargement

# 🔹 Trouver tous les liens des articles
article_links = []
i=2
for link in driver.find_elements(By.CSS_SELECTOR, "#dmk2articlesfilter_results > table:nth-child(i) > tbody > tr > td.tsw > h3"):  # MODIFIER ICI
    # #dmk2articlesfilter_results > table:nth-child(2) > tbody > tr > td.tsw > h3
    # #dmk2articlesfilter_results > table:nth-child(3) > tbody > tr > td.tsw > h3
    # #dmk2articlesfilter_results > table:nth-child(8) > tbody > tr > td.tsw > h3
    href = link.get_attribute("href")
    if href and "article" in href:  # Vérifier que c'est un lien valide
        article_links.append(href)
    i+=1

print(f"🔍 Trouvé {len(article_links)} articles à scraper...")

articles = []

# 🔹 Parcourir chaque article et scraper les infos
for url in article_links:
    try:
        driver.get(url)
        time.sleep(2)  # Attendre le chargement de la page

        # ✅ MODIFIER ICI avec les bons sélecteurs CSS/XPath pour ton site
        title = driver.find_element(By.CSS_SELECTOR, "#k2Container > div.itemHeader > h1").text  # Exemple
        
        date = driver.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")  # Exemple
        author = driver.find_element(By.CSS_SELECTOR, ".author-name").text  # Exemple
        category = driver.find_element(By.CSS_SELECTOR, ".category-name").text  # Exemple
        content = "\n".join([p.text for p in driver.find_elements(By.CSS_SELECTOR, "div.article-content p")])  # Exemple

        articles.append({
            "title": title,
            "date": date,
            "author": author,
            "category": category,
            "content": content,
            "url": url
        })
    except Exception as e:
        print(f"❌ Erreur lors du scraping de {url}: {e}")

# 🔹 Sauvegarde dans un fichier JSON
with open("articles.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

# 🔹 Fermer le navigateur
driver.quit()

print("✅ Scraping terminé avec succès ! 🚀")


In [ ]:
import scrapy
import json
import os
from sentence_transformers import SentenceTransformer

# Initialisation du modèle d'embedding pour la recherche sémantique
model = SentenceTransformer("all-MiniLM-L6-v2")

class ArticleSpider(scrapy.Spider):
    name = "article_spider"
    start_urls = ["https://example.com"]  # Remplacez par l'URL du site à scraper

    def parse(self, response):
        for article in response.css("article"):  # Adapter le sélecteur selon le site
            title = article.css("h1::text, h2::text").get()
            summary = article.css("p::text").get()
            date = article.css("time::attr(datetime)").get()
            author = article.css(".author::text").get()
            category = article.css(".category::text").get()
            content = "\n".join(article.css("p::text").getall())
            url = response.url
            
            # Génération de l'embedding pour la recherche sémantique
            embedding = model.encode(content).tolist()
            
            article_data = {
                "title": title,
                "summary": summary,
                "date": date,
                "author": author,
                "category": category,
                "content": content,
                "url": url,
                "embedding": embedding
            }
            
            yield article_data

# Commande pour exécuter le script et sauvegarder les données
# scrapy runspider scraper.py -o articles.json


In [ ]:
# avec selenuim
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import json
from sentence_transformers import SentenceTransformer
import time

# Initialisation du modèle d'embedding
model = SentenceTransformer("all-MiniLM-L6-v2")

# Configuration du navigateur
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Mode sans interface graphique
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# URL du site à scraper
start_url = "https://example.com"
driver.get(start_url)

# Attendre le chargement de la page
time.sleep(3)

articles = []

# Extraction des articles
article_elements = driver.find_elements(By.TAG_NAME, "article")  # Adapter le sélecteur selon le site
for article in article_elements:
    try:
        title = article.find_element(By.TAG_NAME, "h1").text or article.find_element(By.TAG_NAME, "h2").text
        summary = article.find_element(By.TAG_NAME, "p").text
        date = article.find_element(By.TAG_NAME, "time").get_attribute("datetime")
        author = article.find_element(By.CLASS_NAME, "author").text
        category = article.find_element(By.CLASS_NAME, "category").text
        content = "\n".join([p.text for p in article.find_elements(By.TAG_NAME, "p")])
        url = driver.current_url
        
        # Génération de l'embedding
        embedding = model.encode(content).tolist()
        
        article_data = {
            "title": title,
            "summary": summary,
            "date": date,
            "author": author,
            "category": category,
            "content": content,
            "url": url,
            "embedding": embedding
        }
        
        articles.append(article_data)
    except Exception as e:
        print(f"Erreur lors de l'extraction d'un article: {e}")

# Sauvegarde des données en JSON
with open("articles.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

# Fermeture du navigateur
driver.quit()
